In [ ]:
# import kagglehub
# from transformers import ViTImageProcessor, ViTModel
# from PIL import Image
# import torch
# import os

# # Step 1: Download Animal Faces Dataset
# dataset_path = kagglehub.dataset_download("andrewmvd/animal-faces")
# print("Dataset downloaded at:", dataset_path)

# # Step 2: Load the ViT model and processor
# processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
# model = ViTModel.from_pretrained("google/vit-base-patch16-224")

# # Step 3: Define categories
# categories = ["cat", "dog", "wild"]

# # Step 4: Prepare for processing
# image_embeddings = {}  # Dictionary to store embeddings
# image_counter = 0  # Counter for processed images
# max_images = 600  # Maximum number of images to process

# # Step 5: Loop through categories and images
# for category in categories:
#     category_path = os.path.join(dataset_path, "afhq", "val", category)
#     image_paths = [os.path.join(category_path, img) for img in os.listdir(category_path)]

#     for img_path in image_paths:
#         if image_counter >= max_images:  # Stop once we've processed 600 images
#             break

#         try:
#             # Open the image
#             image = Image.open(img_path).convert("RGB")

#             # Process the image and get embeddings
#             inputs = processor(images=image, return_tensors="pt")
#             outputs = model(**inputs)
#             embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()  # Mean pooling for embeddings

#             assert embeddings.shape == (768,), f"Unexpected embedding shape: {embeddings.shape}"

#             embeddings = embeddings / np.linalg.norm(embeddings)  # Normalize to unit vector

#             # Store the embedding
#             image_embeddings[img_path] = {"embedding": embeddings, "metadata": {"category": category}}
#             image_counter += 1

#             print(f"Processed {image_counter} images...")

#         except Exception as e:
#             print(f"Error processing image {img_path}: {e}")

#     if image_counter >= max_images:  # Break outer loop if 600 images are processed
#         break

# # Step 6: Save embeddings to a file
# if image_embeddings:
#     torch.save(image_embeddings, "animal_faces_embeddings_600_images.pth")
#     print(f"Processed {len(image_embeddings)} images. Embeddings saved to 'animal_faces_embeddings_600_images.pth'.")
# else:
#     print("No embeddings were saved.")


Dataset downloaded at: /root/.cache/kagglehub/datasets/andrewmvd/animal-faces/versions/1


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 1 images...
Processed 2 images...
Processed 3 images...
Processed 4 images...
Processed 5 images...
Processed 6 images...
Processed 7 images...
Processed 8 images...
Processed 9 images...
Processed 10 images...
Processed 11 images...
Processed 12 images...
Processed 13 images...
Processed 14 images...
Processed 15 images...
Processed 16 images...
Processed 17 images...
Processed 18 images...
Processed 19 images...
Processed 20 images...
Processed 21 images...
Processed 22 images...
Processed 23 images...
Processed 24 images...
Processed 25 images...
Processed 26 images...
Processed 27 images...
Processed 28 images...
Processed 29 images...
Processed 30 images...
Processed 31 images...
Processed 32 images...
Processed 33 images...
Processed 34 images...
Processed 35 images...
Processed 36 images...
Processed 37 images...
Processed 38 images...
Processed 39 images...
Processed 40 images...
Processed 41 images...
Processed 42 images...
Processed 43 images...
Processed 44 images.

In [ ]:
import kagglehub
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import os
import numpy as np
import uuid
import requests

# Step 1: Download Animal Faces Dataset
dataset_path = kagglehub.dataset_download("andrewmvd/animal-faces")
print("Dataset downloaded at:", dataset_path)


# Step 2: Load the ViT model and processor
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTModel.from_pretrained("google/vit-base-patch16-224")


# Step 3: Define categories
categories = ["cat", "dog", "wild"]

# Step 4: Prepare for processing
image_embeddings = []  # List to store embeddings and metadata
image_counter = 0  # Counter for processed images
max_images = 600  # Maximum number of images to process

# Normalization function
def normalize_embeddings(embeddings):
    norm = np.linalg.norm(embeddings, ord=2, axis=-1, keepdims=True)
    normalized_embeddings = embeddings / norm
    return normalized_embeddings

# Step 5: Loop through categories and images
for category in categories:
    category_path = os.path.join(dataset_path, "afhq", "val", category)
    image_paths = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    for img_path in image_paths:
        if image_counter >= max_images:  # Stop once we've processed 600 images
            break

        try:
            # Open the image
            image = Image.open(img_path).convert("RGB")

            # Process the image and get embeddings
            inputs = processor(images=image, return_tensors="pt")
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()  # Mean pooling for embeddings

            # Check embedding consistency
            assert embeddings.shape == (768,), f"Unexpected embedding shape: {embeddings.shape}"

            # Normalize embeddings
            normalized_embeddings = normalize_embeddings(embeddings)

            # Create unique ID (can use image path or generate a UUID)
            unique_id = str(uuid.uuid4())  # Generate a unique identifier

            # Store the embedding and metadata (image path, category)
            image_embeddings.append({
                "id": unique_id,
                "vector": normalized_embeddings.tolist(),
                "payload": {"category": category, "image_name": img_path}
            })

            image_counter += 1  # Increment counter
            print(f"Processed {image_counter} images...")

        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    if image_counter >= max_images:  # Break outer loop if 600 images are processed
        break

# Step 6: Save embeddings to a file (Optional)
if image_embeddings:
    torch.save(image_embeddings, "animal_faces_embeddings_600_images.pth")
    print(f"Processed {len(image_embeddings)} images. Embeddings saved to 'animal_faces_embeddings_600_images.pth'.")
else:
    print("No embeddings were saved.")

100%|██████████| 696M/696M [00:05<00:00, 123MB/s] 

Extracting files...


Dataset downloaded at: /root/.cache/kagglehub/datasets/andrewmvd/animal-faces/versions/1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 1 images...
Processed 2 images...
Processed 3 images...
Processed 4 images...
Processed 5 images...
Processed 6 images...
Processed 7 images...
Processed 8 images...
Processed 9 images...
Processed 10 images...
Processed 11 images...
Processed 12 images...
Processed 13 images...
Processed 14 images...
Processed 15 images...
Processed 16 images...
Processed 17 images...
Processed 18 images...
Processed 19 images...
Processed 20 images...
Processed 21 images...
Processed 22 images...
Processed 23 images...
Processed 24 images...
Processed 25 images...
Processed 26 images...
Processed 27 images...
Processed 28 images...
Processed 29 images...
Processed 30 images...
Processed 31 images...
Processed 32 images...
Processed 33 images...
Processed 34 images...
Processed 35 images...
Processed 36 images...
Processed 37 images...
Processed 38 images...
Processed 39 images...
Processed 40 images...
Processed 41 images...
Processed 42 images...
Processed 43 images...
Processed 44 images.

'/root/.cache/kagglehub/datasets/andrewmvd/animal-faces/versions/1'

In [ ]:
# Save Embeddings (Optional)
if image_embeddings:
    torch.save(image_embeddings, "animal_faces_embeddings_subset.pth")
    print(f"Processed images for all categories. Embeddings saved to 'animal_faces_embeddings_subset.pth'.")
else:
    print("No embeddings found, please check image processing.")


Processed images for all categories. Embeddings saved to 'animal_faces_embeddings_subset.pth'.


In [ ]:
# Check embeddings before saving
print(f"Embeddings collected so far: {len(image_embeddings)}")
if image_embeddings:
    print(f"Example image and its embedding: {image_embeddings[0]}")  # Access the first element directly using indexing

Embeddings collected so far: 600
Example image and its embedding: {'id': 'f7aa1287-a85d-4471-bce6-b95edfc99a32', 'vector': [0.015280572697520256, 0.0494522824883461, 0.0139558594673872, -0.023271942511200905, -0.004137521609663963, -0.0571618415415287, 0.008947785012423992, 0.004165568854659796, 0.038465630263090134, -0.06497754901647568, 0.03921692445874214, 0.030588561668992043, 0.015461677685379982, -0.07492119818925858, -0.025983046740293503, 0.03735591471195221, -0.011085517704486847, 0.008814164437353611, 0.0486212894320488, 0.002552485791966319, -0.05365430563688278, 0.07197925448417664, -0.015181999653577805, 0.04862608388066292, -0.02681574411690235, 0.03601277992129326, 0.022710133343935013, 0.033558908849954605, 0.0034014114644378424, -0.006066921167075634, 0.01116890087723732, 0.03796723857522011, 0.029398735612630844, 0.0013641983969137073, 0.02702774666249752, 0.018467575311660767, -0.04983597621321678, 0.006594264879822731, -0.026725446805357933, -0.0140724778175354, 0.0

In [ ]:
print(f"Processing category: {category}")
print(f"Processing image: {image_path}")


Processing category: dog


NameError: name 'image_path' is not defined

In [ ]:
import numpy as np

def validate_embeddings(embeddings):
    """
    Validates the embeddings before storing them in QDrant.
    Checks for:
    1. Dimensionality consistency
    2. Normalization of embeddings (unit vectors)
    3. Correct data format
    """
    # 1. Check consistency of embedding dimensions
    expected_dim = 768  # Your vector size
    for entry in embeddings:
        embedding = np.array(entry["vector"])

        # Check if the shape matches the expected dimension
        if embedding.shape != (expected_dim,):
            print(f"Embedding for ID {entry['id']} has an incorrect shape: {embedding.shape}. Expected shape: ({expected_dim},)")
            return False

        # 2. Check for normalization (unit vector check)
        norm = np.linalg.norm(embedding, ord=2)
        if not np.isclose(norm, 1.0):
            print(f"Embedding for ID {entry['id']} is not normalized. Norm: {norm}")
            return False

        # 3. Check if the payload has necessary fields (you can modify this based on your needs)
        if "category" not in entry["payload"] or "image_name" not in entry["payload"]:
            print(f"Embedding for ID {entry['id']} is missing category or image_name in its payload.")
            return False

    print("All embeddings are validated successfully!")
    return True

# Example usage:
# Validate the embeddings before storing them in QDrant
if validate_embeddings(image_embeddings):
    print("Embeddings are valid and ready for storage in QDrant.")
else:
    print("Some embeddings are invalid and cannot be stored in QDrant.")


All embeddings are validated successfully!
Embeddings are valid and ready for storage in QDrant.


In [ ]:
print(image_embeddings[:1])  # Print the first item


[{'id': 'f7aa1287-a85d-4471-bce6-b95edfc99a32', 'vector': [0.015280572697520256, 0.0494522824883461, 0.0139558594673872, -0.023271942511200905, -0.004137521609663963, -0.0571618415415287, 0.008947785012423992, 0.004165568854659796, 0.038465630263090134, -0.06497754901647568, 0.03921692445874214, 0.030588561668992043, 0.015461677685379982, -0.07492119818925858, -0.025983046740293503, 0.03735591471195221, -0.011085517704486847, 0.008814164437353611, 0.0486212894320488, 0.002552485791966319, -0.05365430563688278, 0.07197925448417664, -0.015181999653577805, 0.04862608388066292, -0.02681574411690235, 0.03601277992129326, 0.022710133343935013, 0.033558908849954605, 0.0034014114644378424, -0.006066921167075634, 0.01116890087723732, 0.03796723857522011, 0.029398735612630844, 0.0013641983969137073, 0.02702774666249752, 0.018467575311660767, -0.04983597621321678, 0.006594264879822731, -0.026725446805357933, -0.0140724778175354, 0.0057914406061172485, -0.028831705451011658, 0.016287175938487053, 

# Qdrant API base URL and API key
QDRANT_URL = "https://88b7f5e2-c78f-4972-b278-fb4d08efb2d1.europe-west3-0.gcp.cloud.qdrant.io:6333"

API_KEY = "hNPv5JhJ5Anz1ximwxkFU7-eAEJ-kuvHAUvN-xaugCBsdc2NFRrwIg"


In [ ]:
import requests

# Replace with your actual Qdrant URL and API key
QDRANT_URL = "https://88b7f5e2-c78f-4972-b278-fb4d08efb2d1.europe-west3-0.gcp.cloud.qdrant.io:6333"
API_KEY = "hNPv5JhJ5Anz1ximwxkFU7-eAEJ-kuvHAUvN-xaugCBsdc2NFRrwIg"

def create_collection(collection_name="ViT", vector_size=768, distance_metric="Cosine"):
    """
    Creates a collection in Qdrant for storing image embeddings and metadata.
    Args:
        collection_name (str): Name of the collection to create.
        vector_size (int): Dimensionality of the vectors (e.g., 768 for ViT embeddings).
        distance_metric (str): Similarity metric to use ("Cosine", "Euclid", etc.).
    """
    payload = {
        "vectors": {
            "size": vector_size,
            "distance": distance_metric  # Use "Cosine" for cosine similarity
        },
        "payload_schema": {
            "image_name": {"type": "keyword"},  # Metadata field for image name
            "category": {"type": "keyword"}     # Metadata field for category (e.g., cat, dog, wild)
        }
    }
    headers = {"api-key": API_KEY, "Content-Type": "application/json"}  # Add your API key if required

    # API call to create the collection
    response = requests.put(f"{QDRANT_URL}/collections/{collection_name}", json=payload, headers=headers)

    # Handle response
    if response.status_code == 200:
        print(f"Collection '{collection_name}' created successfully.")
    else:
        print(f"Failed to create collection: {response.status_code}, {response.json()}")

# Call the function to create the collection
create_collection()


Failed to create collection: 409, {'status': {'error': 'Wrong input: Collection `ViT` already exists!'}, 'time': 0.022532832}


In [ ]:
# import requests
# import numpy as np

# COLLECTION_NAME="ViT"
# QDRANT_URL = "https://88b7f5e2-c78f-4972-b278-fb4d08efb2d1.europe-west3-0.gcp.cloud.qdrant.io:6333"
# API_KEY = "hNPv5JhJ5Anz1ximwxkFU7-eAEJ-kuvHAUvN-xaugCBsdc2NFRrwIg"
# VECTOR_SIZE = 768

# def store_in_qdrant(embeddings, collection_name=COLLECTION_NAME):
#     """
#     Stores the validated embeddings into the QDrant collection.
#     """
#     # Prepare the list of vectors and payloads
#     # vectors = []
#     points = []

#     for data in embeddings:
#         # Prepare the vector and payload
#         vector = data["vector"]
#         metadata = data["payload"]

#         # Prepare the payload with image_name and category
#         payload = {
#             "image_name": metadata["image_name"],
#             "category": metadata["category"]
#         }

#         # Append the vector and payload
#         points.append({
#             "id": image_path,  # Unique identifier for the vector
#             "vector": vector,  # Convert numpy array to list
#             "payload": payload
#         })

#     # Prepare the request payload for QDrant
#     payload = {
#         "points": points
#     }

#     headers = {
#         "api-key": API_KEY,
#         "Content-Type": "application/json"
#     }

#     # Send request to QDrant to insert the vectors
#     response = requests.put(f"{QDRANT_URL}/collections/{collection_name}/points", json=payload, headers=headers)

#     if response.status_code == 200:
#         print(f"Successfully stored {len(vectors)} vectors in the '{collection_name}' collection.")
#     else:
#         print(f"Failed to store vectors: {response.status_code} - {response.json()}")

# # Now store the embeddings in QDrant
# store_in_qdrant(image_embeddings)


import requests

COLLECTION_NAME = "ViT"
QDRANT_URL = "https://88b7f5e2-c78f-4972-b278-fb4d08efb2d1.europe-west3-0.gcp.cloud.qdrant.io:6333"
API_KEY = "hNPv5JhJ5Anz1ximwxkFU7-eAEJ-kuvHAUvN-xaugCBsdc2NFRrwIg"

def store_in_qdrant(embeddings, collection_name=COLLECTION_NAME):
    """
    Stores embeddings in QDrant with proper validation.
    """
    points = []

    for data in embeddings:
        try:
            # Extract the vector, id, and optional payload
            vector = data["vector"]
            point_id = data["id"]  # Ensure this is a valid UUID or integer

            # Optional payload - modify according to your dataset
            payload = data.get("payload", {"image_name": "unknown", "category": "unknown"})

            # Prepare point for QDrant
            points.append({
                "id": point_id,  # Use the provided unique ID
                "vector": vector,  # Ensure vector is a list
                "payload": payload  # Include metadata as payload
            })
        except KeyError as e:
            print(f"Missing key {e} in data: {data}")
        except Exception as e:
            print(f"Error processing data: {data}, Error: {e}")

    if points:
        # Prepare request payload
        payload = {"points": points}
        headers = {
            "api-key": API_KEY,
            "Content-Type": "application/json"
        }

        # API call to QDrant
        response = requests.put(f"{QDRANT_URL}/collections/{collection_name}/points", json=payload, headers=headers)

        # Handle response
        if response.status_code == 200:
            print(f"Successfully stored {len(points)} vectors in the '{collection_name}' collection.")
        else:
            print(f"Failed to store vectors: {response.status_code} - {response.json()}")
    else:
        print("No valid points to store in QDrant.")

# Call the function
store_in_qdrant(image_embeddings)


Successfully stored 600 vectors in the 'ViT' collection.


In [ ]:
def search_in_qdrant(query_vector, collection_name=COLLECTION_NAME, top_k=10, similarity_threshold=0.8, ef=128):
    """
    Search for the most similar vectors in QDrant.

    Parameters:
        query_vector (list): The vector to search with.
        collection_name (str): Name of the collection to search in.
        top_k (int): Number of top results to return.
        similarity_threshold (float): Minimum similarity score to consider a result relevant.
        ef (int): Value for the ef parameter to improve search quality.

    Returns:
        list: A list of matching vectors with similarity scores and metadata.
    """
    headers = {
        "api-key": API_KEY,
        "Content-Type": "application/json"
    }

    # Payload for search
    payload = {
        "vector": query_vector,  # Query vector for similarity search
        "top": top_k,  # Number of top results to retrieve
        "params": {
            "hnsw_ef": ef  # Set ef for better recall during search
        }
    }

    # Perform the search
    response = requests.post(f"{QDRANT_URL}/collections/{collection_name}/points/search", json=payload, headers=headers)

    if response.status_code == 200:
        results = response.json()

        # Filter results based on similarity threshold
        filtered_results = [
            {
                "id": result["id"],
                "score": result["score"],  # Cosine similarity score
                "payload": result.get("payload", {})
            }
            for result in results if result["score"] >= similarity_threshold
        ]

        print(f"Search returned {len(filtered_results)} results with similarity >= {similarity_threshold}.")
        return filtered_results

    else:
        print(f"Failed to search: {response.status_code} - {response.json()}")
        return []



In [ ]:
from PIL import Image
import numpy as np
import torch
from torchvision import transforms

def preprocess_image(image_path, image_size=224):
    """
    Preprocess the query image to match the stored embeddings' preprocessing steps.

    Parameters:
        image_path (str): Path to the query image.
        image_size (int): The size to which the image is resized (default: 224x224).

    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    try:
        # Define transformations (consistent with embedding generation)
        preprocess = transforms.Compose([
            transforms.Resize((image_size, image_size)),  # Resize to target size
            transforms.ToTensor(),  # Convert to tensor
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
        ])

        # Load image and apply transformations
        image = Image.open(image_path).convert("RGB")  # Ensure 3 channels
        return preprocess(image).unsqueeze(0)  # Add batch dimension

    except Exception as e:
        print(f"Error in image preprocessing: {e}")
        return None


def generate_embedding(query_image, model):
    """
    Generate the embedding for the query image using the ViT model.

    Parameters:
        query_image (torch.Tensor): Preprocessed query image.
        model (torch.nn.Module): ViT model for embedding generation.

    Returns:
        np.array: Normalized embedding vector (unit length).
    """
    try:
        # Set the model to evaluation mode
        model.eval()

        with torch.no_grad():
            embedding = model(query_image).numpy().flatten()  # Generate embedding and flatten

        # Normalize to unit length for cosine similarity
        normalized_embedding = embedding / np.linalg.norm(embedding)
        return normalized_embedding

    except Exception as e:
        print(f"Error in embedding generation: {e}")
        return None


def query_qdrant(image_path, model, collection_name=COLLECTION_NAME, top_k=10, similarity_threshold=0.8, ef=128):
    """
    Query QDrant to find images similar to the input query image.

    Parameters:
        image_path (str): Path to the query image.
        model (torch.nn.Module): ViT model for generating embeddings.
        collection_name (str): Name of the QDrant collection.
        top_k (int): Number of top results to return.
        similarity_threshold (float): Minimum similarity score for relevant results.
        ef (int): Exploration factor for better recall.

    Returns:
        list: A list of results containing IDs, scores, and metadata.
    """
    try:
        # Step 1: Preprocess the image
        query_image = preprocess_image(image_path)
        if query_image is None:
            return "Invalid query image. Please check the input."

        # Step 2: Generate the query embedding
        query_embedding = generate_embedding(query_image, model)
        if query_embedding is None:
            return "Error generating query embedding. Please try again."

        # Step 3: Query QDrant
        headers = {
            "api-key": API_KEY,
            "Content-Type": "application/json"
        }
        payload = {
            "vector": query_embedding.tolist(),  # Convert to list for JSON compatibility
            "top": top_k,
            "params": {
                "hnsw_ef": ef  # Exploration factor for better recall
            }
        }

        # API call
        response = requests.post(f"{QDRANT_URL}/collections/{collection_name}/points/search", json=payload, headers=headers)

        if response.status_code == 200:
            results = response.json()

            # Step 4: Filter results by similarity threshold
            filtered_results = [
                {
                    "id": result["id"],
                    "score": result["score"],
                    "payload": result.get("payload", {})
                }
                for result in results if result["score"] >= similarity_threshold
            ]

            if not filtered_results:
                return f"No results found with similarity >= {similarity_threshold}."

            # Step 5: Sort results by similarity score
            filtered_results.sort(key=lambda x: x["score"], reverse=True)
            return filtered_results

        else:
            return f"Failed to query QDrant: {response.status_code} - {response.json()}"

    except Exception as e:
        return f"Error during query operation: {e}"



In [ ]:
# Example image query
query_image_path = "/content/flickr_cat_000002.jpg"

# Pretrained ViT model (ensure it's loaded with the same weights as used during embedding generation)
model = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8')

# Query
results = query_qdrant(query_image_path, model, top_k=5, similarity_threshold=0.85, ef=256)

# Display results
if isinstance(results, list):
    for idx, result in enumerate(results, 1):
        print(f"Result {idx}:")
        print(f" - ID: {result['id']}")
        print(f" - Similarity Score: {result['score']}")
        print(f" - Metadata: {result['payload']}")
else:
    print(results)


Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_vitbase8_pretrain.pth
100%|██████████| 327M/327M [00:03<00:00, 101MB/s]


Error during query operation: string indices must be integers
